# Particle Swarm Optimization Code Documentation

Our implementation of PSO relies on two fundamental abstractions (a _solver_ objet which manipulates a collection of _particles_) which are particularly suited to the Object-Oriented Programing (OOP) paradigm.

We implement the folowing architecture in the `PSO` module (see the `PSO.py` file):

![title](PSO API Doc.jpg)

Here is an overview of the classes we implemented. The code in PSO.py and every function is properly documented.

## The `Particle` class

### Attributes
We implement a Particle as having the following __attributes__:

- a `position`: the current position of the particle. Initiated uniformly at random in the hypercube $[\texttt{lower}, \texttt{upper}]^{\texttt{ndim}}$ where `lower`, `upper`, and `ndim` are provided by the user during class instantation. Implemented as a `numpy` array.

- a `velocity`: the current velocity of the particle. Initiated as a random requence of -1s and 1s accross `ndim` dimensions. Implemented as a `numpy` array.

- a `personal_best_position` and associated `personal_best_fitness` to keep track of a particle best known position and fitness/error (essentially the particle's memory). Initiated to the particle's original (random) position with an error of positive infinity.

- `c1 = 1.49618`, `c2 = 1.49618`, `w = 0.7298`: the cognitive, social, and inertia parameters. User-provided, with defaults values picked following Van den Bergh and Engelbrecht (2006).


### Methods

A Particle has the following __methods__:

- `update_velocity(self, global_best)`: given the global best known position, update the particle's velocity according to the PSO rule.

- `move(self)`: move the particle to a new position using the update rule $position_{t+1} = position_{t} + velocity_{t}$. By default, the particles are allowed to explore the search space beyond the initialisation boundaries. To avoid this behaviour, the user should adapt the target function by setting out-of-bound values to positive infinity (the induced discontinuity will not affect the algorithm's performance as it is derivative-free)

## The `PSO` base class

A `PSO` object is a solver that implements the PSO algorithm to minimize a given function. It is instantiated as follows:

```python
PSO(num_particles, function, n_iter, ndim, lower = -10, upper = 10,
    c1 = 1.49618, c2 = 1.49618, w = 0.7298, epsilon = 10e-7)
```


### Attributes

- `particles`: a `list` of `Particle` objects initiated as described above. There are `num_particles` particles.

- `fitnesses`: a `num_particles`-dimensional array containing the current fitness (aka value) of each of the particles (This does not exist for Asynchronous Parallel PSO)

- `global_best` and associated `global_best_fitness`: the global best known position along with the function value at that point.

- `function`: the function to be optimised. Of the form `f(Array:x) => Float:y`.

- `n_iter`: the number of iterations to run the PSO algorithm for. Since this number doesn't really make sense in the asynchronous case, we use a `n_func_eva` attribute instead which is computer as `num_particles * n_iter`.

- `epsilon`: defines convergence. If an update between two consecutive global best values is smaller than `epsilon` then the algorithm is said to have converged. This is not implemented for the Asynchronous case.

### Methods

- `__init__` and `__str__` to instatiate and print a PSO object.

- `get_fitnesses(self)`: evaluate all particle's fitnesses. It is done differently depending on whether the algorithm runs in parallel or not.

- `update_particles(self)`: updates each particle's best known position if it's current fitness is better than its previous best.

- `update_best(self)`: get new best position and associated fitness, if applicable.

- `move_particles(self)`: run one iteration of the PSO algorithm. Update particles' velocities and makes them move.

- `run(self, verbose=True)`: runs the algorithm for a prespecified number of steps, or until a convergence criterion is reached. Returns the algorithm's global best position. The `verbose` argument makes the output a little bit more informative.

## The PSO_synchron class

This solver is derived from the base PSO class and additionally has a `pooler` attribute which is how the `multiprocessing` library implements a CPU cluster for (synchronous) parallel computing. The number of CPUs used is set to the number of CPUs on your machine minus 1.

The `get_fitnesses` method is implemented so as to make use of the pooler and thus run in parallel.

## The PSO_asynchron class

This solver is derived from the base PSO class with several major differences.

- TOBECOMPLETED

The asynchronous parallelism is based on a `Queue` 

In [ ]:
        # Create shared memory array for storing global best position
        self.global_best = Array('d', ndim)
        # Create shared memory Value for global best fitness
        self.global_best_fitness = Value('d', np.inf)
        # Shared counter (to count the number of function evaluations)
        self.count = Value('i', 0)
        # How many time should we evaluate the function?
        self.n_func_eva = n_iter * num_particles
        self.stop_queuing = self.n_func_eva - num_particles
    

### Example Use

IN:
```python
from PSO import Particle, PSO, PSO_synchron, PSO_asynchon
from functions import quad_function

# instantiate the solver
solver_basic = PSO(num_particles = 20, function = quad_function, n_iter = 200, ndim = 2,
             lower = -10, upper = 10, epsilon = 0)
solver_basic.run()
```
----
OUT:


```
Running the PSO algorithm with 20 particles, for at most 200 iterations.

After 200 iterations,
Found minimum at [2.  0.5] with value 1.684771319605328e-19.
array([2. , 0.5])
```